# 파이썬 5월 23일 강의 복습
> "7월 9일"

- toc:true
- branch: master
- badges: true
- comments: true
- author: 최석진
- categories: [Python]

## 클래스 공부 4단계

In [1]:
import numpy as np

### motivating Example

`-` 가위바위보

In [4]:
class RPC:
    def __init__(self,candidate=['가위','바위','보']):
        self.candidate=candidate
    def throw(self):
        print(np.random.choice(self.candidate))

In [5]:
a=RPC() #__init__는 암묵적으로 실행

In [6]:
a.throw()

바위


`-` throw = choose + show 의 느낌

In [7]:
class RPC: ##시점1
    def __init__(self,candidate=['가위','바위','보']):
        self.candidate=candidate
    def choose(self):
        self.actions=np.random.choice(self.candidate)
    def show(self):
        print(self.actions)

In [8]:
a=RPC() ## 시점2

In [9]:
a.actions ## 시점3

AttributeError: 'RPC' object has no attribute 'actions'

In [11]:
a.choose() ##시점4

In [12]:
a.actions ## 시점5

'보'

In [15]:
a.show() ## 시점6

보


*보충학습*: 위와 같은 코드

In [16]:
class _RPC: ##시점1
    pass # 빈 클래스를 정의하는 방법

In [17]:
_a=_RPC() ##시점2
def _init(_a,candidate=['가위','바위','보']):
    _a.candidate=candidate
_init(_a)

In [18]:
_a.actions ##시점3

AttributeError: '_RPC' object has no attribute 'actions'

In [19]:
def _choose(_a): ##시점4
    _a.actions=np.random.choice(_a.candidate)
_choose(_a)

In [21]:
_a.actions ##시점5

'바위'

In [23]:
def _show(_a): ##시점6
    print(_a.actions)
_show(_a)

바위


`-` 또 다른 인스턴스 b를 만듬. b는 가위만 낼 수 있음

In [24]:
b=RPC(['가위'])

In [25]:
b.candidate

['가위']

In [26]:
b.choose()
b.show()

가위


`-`  a,b의 선택들을 모아서 기록하고 싶음

In [27]:
class RPC:
    def __init__(self,candidate=['가위','바위','보']):
        self.candidate=candidate
        self.actions=list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])

In [28]:
a=RPC()
b=RPC(['가위'])

In [29]:
for i in range(5):
    a.choose()
    a.show()

바위
보
바위
바위
가위


In [30]:
a.actions

['바위', '보', '바위', '바위', '가위']

In [31]:
for i in range(5):
    b.choose()
    b.show()

가위
가위
가위
가위
가위


In [32]:
b.actions

['가위', '가위', '가위', '가위', '가위']

`-` info라는 함수를 만들어 a,b오브젝트가 가지고 있는 정보를 모두 보도록 함.

In [33]:
class RPC:
    def __init__(self,candidate=['가위','바위','보']):
        self.candidate=candidate
        self.actions=list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    def info(self):
        print("낼 수 있는 패 : {} \n기록:{}".format(self.candidate,self.actions))

In [34]:
a=RPC()
b=RPC(['가위'])

In [35]:
for i in range(5):
    a.choose()
    a.show()

가위
보
보
보
바위


In [36]:
for i in range(5):
    b.choose()
    b.show()

가위
가위
가위
가위
가위


In [37]:
a.info()

낼 수 있는 패 : ['가위', '바위', '보'] 
기록:['가위', '보', '보', '보', '바위']


In [38]:
b.info()

낼 수 있는 패 : ['가위'] 
기록:['가위', '가위', '가위', '가위', '가위']


`-` 만들고 보니 info와 print의 기능이 유사함 $\to$ print(a)를 하면 a.info()와 동일한 효과를 내도록 만들 수 있을까?

`-` 말도 안되는 소리 같음 왜?
- 이유1 : print는 파이썬 내장 기능, 내장기능을 우리가 맘대로 커스터마이징해서 쓰기는 어려울 것 같음
- 이유2 : 이유1이 해결된다고 하더라도 우리가 그동안 사용했던 수많은 print()의 결과는 어떻게 되는가?

`-` 그런데 a의 자료형 (RPC 자료형)에 해당하는 오브젝트들에 한정하여 print를 수정하는 방법이 가능하다면? (그럼 다른 오브젝트들은 수정된 print에 영향을 받지 않음)

### **`__str__`**

`-` 관찰1 : 현재 print(a)의 결과는 아래와 같다.

In [39]:
print(a)

- a는 RPC클래스에서 만든 오브젝트이며 a가 저장된 메모리 주소는 0x7fd8403d36d0라는 의미

`-` 관찰2 : a에는 `__str__`이 있다.

In [41]:
set(dir(a)) & {'__str__'}

{'__str__'}

이것을 함수처럼 사용하니까 아래와 같이 된다.

In [42]:
a.__str__()

'<__main__.RPC object at 0x7fd8403d36d0>'

? print(a) 와 동일한 결과값이 출력

`-` 생각 : 만약에 내가 `a.__str__()`라는 함수를 재정의해서 리턴값을 임의로 바꾸게 되면 print(a)해서 나오는 결과는 어떻게 될까?

(예비학습) 함수 덮어씌우기

In [43]:
def f():
    print('asdf')

In [44]:
f()

asdf


In [45]:
def f():
    print('Seokjin')

In [46]:
f()

Seokjin


이런식으로 함수가 이미 정의되어 있다하더라도, 내가 나중에 덮어씌우면 그 함수의 기능을 다시 정의한다.

In [53]:
class RPC:
    def __init__(self,candidate=['가위','바위','보']):
        self.candidate=candidate
        self.actions=list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    def __str__(self):
        return 'Seokjin'
    def info(self):
        print("낼 수 있는 패: {}\n기록: {}".format(self.candidate,self.actions))

In [48]:
a=RPC()

In [50]:
print(a)

Seokjin


In [51]:
print(a.__str__())

Seokjin


`-` `__str__`의 리턴값을 info에서 타이핑했던 문자열로 재정의 한다면??

In [54]:
class RPC:
    def __init__(self,candidate=['가위','바위','보']):
        self.candidate=candidate
        self.actions=list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    def __str__(self):
        return "낼 수 있는 패: {}\n기록: {}".format(self.candidate,self.actions)

In [55]:
a=RPC()

In [56]:
print(a)

낼 수 있는 패: ['가위', '바위', '보']
기록: []


In [57]:
a.choose()
a.show()

보


In [58]:
print(a)

낼 수 있는 패: ['가위', '바위', '보']
기록: ['보']


### 파이썬의 비밀2

`-` `print(a)`와 `print(a.__str__())`는 같은 문법.

`-` `a.__str__()`와 `str(a)`도 같은 문법.

In [59]:
a.__str__()

"낼 수 있는 패: ['가위', '바위', '보']\n기록: ['보']"

In [60]:
str(a)

"낼 수 있는 패: ['가위', '바위', '보']\n기록: ['보']"

### `__repr__`

`-` 생각해보니 print를 해서 우리가 원하는 정보를 확인하는 것은 아니었음

In [61]:
a=[1,2,3]

In [62]:
a

[1, 2, 3]

In [63]:
print(a)

[1, 2, 3]


`-` `a+엔터`는 `print(a)+엔터`와 같은 효과?

(반례)

In [64]:
a=np.array([1,2,3,4]).reshape(2,2)

In [65]:
a

array([[1, 2],
       [3, 4]])

In [66]:
print(a)

[[1 2]
 [3 4]]


`-` 서로 다른 경우도 있음 $\to$ 추측: 서로 다른 숨겨진 기능이 있다! $\to$ 결론: 추측이 맞음. 그 기능은 `__repr__`에 저장되어있음

In [69]:
class RPC:
    def __init__(self,candidate=['가위','바위','보']):
        self.candidate=candidate
        self.actions=list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    def __repr__(self):
        return "낼 수 있는 패: {}\n기록: {}".format(self.candidate,self.actions)

In [70]:
a=RPC()

In [71]:
a # print(a.__repr__())

낼 수 있는 패: ['가위', '바위', '보']
기록: []

In [72]:
a=np.array([1,2,3])

In [73]:
a

array([1, 2, 3])

In [74]:
print(a)

[1 2 3]


In [75]:
a.__repr__()

'array([1, 2, 3])'

In [76]:
a.__str__()

'[1 2 3]'

### 파이썬의 비밀3

`-` 대화형콘솔에서 `오브젝트이름+엔터`를 쳐서 나오는 출력은 `__repr__`의 결과와 연관있다.

In [77]:
a=np.array(range(10000)).reshape(100,100)
a

array([[   0,    1,    2, ...,   97,   98,   99],
       [ 100,  101,  102, ...,  197,  198,  199],
       [ 200,  201,  202, ...,  297,  298,  299],
       ...,
       [9700, 9701, 9702, ..., 9797, 9798, 9799],
       [9800, 9801, 9802, ..., 9897, 9898, 9899],
       [9900, 9901, 9902, ..., 9997, 9998, 9999]])

In [78]:
a.__repr__()

'array([[   0,    1,    2, ...,   97,   98,   99],\n       [ 100,  101,  102, ...,  197,  198,  199],\n       [ 200,  201,  202, ...,  297,  298,  299],\n       ...,\n       [9700, 9701, 9702, ..., 9797, 9798, 9799],\n       [9800, 9801, 9802, ..., 9897, 9898, 9899],\n       [9900, 9901, 9902, ..., 9997, 9998, 9999]])'

In [79]:
repr(a)

'array([[   0,    1,    2, ...,   97,   98,   99],\n       [ 100,  101,  102, ...,  197,  198,  199],\n       [ 200,  201,  202, ...,  297,  298,  299],\n       ...,\n       [9700, 9701, 9702, ..., 9797, 9798, 9799],\n       [9800, 9801, 9802, ..., 9897, 9898, 9899],\n       [9900, 9901, 9902, ..., 9997, 9998, 9999]])'

`__repr__`와 `__str__`의 우선적용 순위

(예제1)

In [84]:
class RPC:
    def __init__(self,candidate=['가위','바위','보']):
        self.candidate=candidate
        self.actions=list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    def __repr__(self):
        return "낼 수 있는 패: {}\n기록: {}".format(self.candidate,self.actions)

In [85]:
a=RPC()
a

낼 수 있는 패: ['가위', '바위', '보']
기록: []

In [86]:
a.__repr__()

"낼 수 있는 패: ['가위', '바위', '보']\n기록: []"

In [87]:
repr(a)

"낼 수 있는 패: ['가위', '바위', '보']\n기록: []"

In [88]:
print(a) #print(a.__str__())

낼 수 있는 패: ['가위', '바위', '보']
기록: []


In [89]:
a.__str__()

"낼 수 있는 패: ['가위', '바위', '보']\n기록: []"

In [90]:
str(a)

"낼 수 있는 패: ['가위', '바위', '보']\n기록: []"

`-` `__str__()`은 건드린적이 없는데 기능이 바뀜

In [91]:
a.__str__??

Signature:      a.__str__()
Call signature: a.__str__(*args, **kwargs)
Type:           method-wrapper
String form:    <method-wrapper '__str__' of RPC object at 0x7fd8207a6e80>
Docstring:      Return str(self).


In [92]:
a.__repr__??

Signature: a.__repr__()
Docstring: Return repr(self).
Source:   
    def __repr__(self):
        return "낼 수 있는 패: {}\n기록: {}".format(self.candidate,self.actions)
File:      /var/folders/l1/qgzhh1fs7tsg8pkmzspwgkyw0000gn/T/ipykernel_52927/1167939066.py
Type:      method


(예제2)

In [93]:
class RPC:
    def __init__(self,candidate=['가위','바위','보']):
        self.candidate=candidate
        self.actions=list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    def __str__(self):
        return "낼 수 있는 패: {}\n기록: {}".format(self.candidate,self.actions)

In [94]:
a=RPC()

In [95]:
print(a)

낼 수 있는 패: ['가위', '바위', '보']
기록: []


In [96]:
a

In [97]:
a.__str__()

"낼 수 있는 패: ['가위', '바위', '보']\n기록: []"

In [98]:
a.__repr__()

'<__main__.RPC object at 0x7fd8408effd0>'

In [101]:
a.__str__??

Signature: a.__str__()
Docstring: Return str(self).
Source:   
    def __str__(self):
        return "낼 수 있는 패: {}\n기록: {}".format(self.candidate,self.actions)
File:      /var/folders/l1/qgzhh1fs7tsg8pkmzspwgkyw0000gn/T/ipykernel_52927/576896846.py
Type:      method


In [102]:
a.__repr__??

Signature:      a.__repr__()
Call signature: a.__repr__(*args, **kwargs)
Type:           method-wrapper
String form:    <method-wrapper '__repr__' of RPC object at 0x7fd8408effd0>
Docstring:      Return repr(self).


(예제3)

In [103]:
class RPC:
    def __init__(self,candidate=['가위','바위','보']):
        self.candidate=candidate
        self.actions=list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    def __repr__(self):
        return "seokjin"
    def __str__(self):
        return "낼 수 있는 패: {}\n기록: {}".format(self.candidate,self.actions)

In [104]:
a=RPC()

In [105]:
a

seokjin

In [106]:
print(a)

낼 수 있는 패: ['가위', '바위', '보']
기록: []


`-` `__str__` & `_repr__`을 건드리지 않고 출력결과를 바꾸고 싶다면?

In [107]:
class RPC:
    def __init__(self,candidate=['가위','바위','보']):
        self.candidate=candidate
        self.actions=list()
    def choose(self):
        self.actions.append(np.random.choice(self.candidate))
    def show(self):
        print(self.actions[-1])
    def _repr_html_(self):
        html_str="""
        낼 수 있는 패 : {} <br/>
        기록 : {}
        """
        return html_str.format(self.candidate,self.actions)

In [108]:
a=RPC()

In [109]:
str(a)

'<__main__.RPC object at 0x7fd8408ef2e0>'

In [110]:
repr(a)

'<__main__.RPC object at 0x7fd8408ef2e0>'

In [111]:
a

In [112]:
for i in range(5):
    a.choose()
    a.show()

가위
보
바위
가위
보


In [113]:
a